In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Im using RandomForest classifier but it works with other types of classifiers as well

In [ ]:
from sklearn.ensemble import RandomForestClassifier

As training data im using reuters-21578 text categorization collection which can be found here: https://www.kaggle.com/weipengfei/ohr8r52

In [ ]:
train = pd.read_csv('../input/ohr8r52/r8/r8-train-stemmed.csv')
test = pd.read_csv('../input/ohr8r52/r8/r8-test-stemmed.csv')
train.head()

I define the GloveVectorizer class similar to the configuration of Sickit Lern classes: init, fit and transform functions.
the Glove model can be found here: https://www.kaggle.com/watts2/glove6b50dtxt

In [ ]:
class GloveVectorizer:
  def __init__(self):
    # load in pre-trained word vectors
    print('Loading word vectors...')
    word2vec = {}
    embedding = []
    idx2word = []
    with open('../input/glove6b50dtxt/glove.6B.50d.txt',encoding="utf8") as f:
      # is just a space-separated text file in the format:
      # word vec[0] vec[1] vec[2] ...
      for line in f:
        values = line.split()
        word = values[0]
        vec = np.asarray(values[1:], dtype='float32')
        word2vec[word] = vec
        embedding.append(vec)
        idx2word.append(word)
    print('Found %s word vectors.' % len(word2vec))

    self.word2vec = word2vec
    self.embedding = np.array(embedding)
    self.word2idx = {v:k for k,v in enumerate(idx2word)}
    self.V, self.D = self.embedding.shape

  def fit(self, data):
    pass

  def transform(self, data):
    X = np.zeros((len(data), self.D))
    n = 0
    emptycount = 0
    for sentence in data:
      tokens = sentence.lower().split()
      vecs = []
      for word in tokens:
        if word in self.word2vec:
          vec = self.word2vec[word]
          vecs.append(vec)
      if len(vecs) > 0:
        vecs = np.array(vecs)
        X[n] = vecs.mean(axis=0)
      else:
        emptycount += 1
      n += 1
    print("Numer of samples with no words found: %s / %s" % (emptycount, len(data)))
    return X

  def fit_transform(self, data):
    self.fit(data)
    return self.transform(data)

we creating GloveVectorizer object and fit-transforms the glove model on the input data

In [ ]:
vectorizer = GloveVectorizer()
Xtrain = vectorizer.fit_transform(train.text)
Ytrain = train.intent

Xtest = vectorizer.transform(test.text)
Ytest = test.intent

create the model, train it, print scores

In [ ]:
model = RandomForestClassifier(n_estimators=200)
model.fit(Xtrain, Ytrain)
print("train score:", model.score(Xtrain, Ytrain))
print("test score:", model.score(Xtest, Ytest))